In [ ]:
import sys
from pathlib import Path
cwd = Path.cwd()
if (cwd / 'src').exists():
    ROOT = cwd
else:
    ROOT = cwd.parent
if str(ROOT) not in sys.path:
    sys.path.insert(0, str(ROOT))
if str(ROOT / 'src') not in sys.path:
    sys.path.insert(0, str(ROOT / 'src'))

import importlib
importlib.invalidate_caches()

# Install pysindy if needed: `pip install pysindy`
import numpy as np
print("Add code to prepare X matrix and run SINDy")